In [71]:
Pkg.add("DataFrames")
using DataFrames, CSV, Statistics;

 Resolving package versions...
  Updating `~/.julia/environments/v1.1/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.1/Manifest.toml`
 [no changes]


In [67]:
df = CSV.read("points.csv");
m = length(df.x);
# TODO: Use zip method?
z = [[ Float64.(df.x[i]), Float64.(df.y[i])] for i=1:m];

In [62]:
using ForwardDiff
# Where x = [xc, yc, r]
r(x) = [
    (x[1] - z[1])^2 + (x[2] - z[2])^2 - x[3]^3 
    for i=1:m
    ]
# Manually computed gradient
A(x) = [
    ( 2*(x[1] - z[1]), 2*(x[2] - z[2]), -2*x[3] )
    for i=1:m
]
# A(x) = ForwardDiff.jacobian(r,x)

A (generic function with 1 method)

In [66]:
function gn(r, A, x, tol=1e-6; maxits=5length(x))
    trace = Matrix(0,2)   # store pair ||A'r||,||x(k+1) - x(k)||
    xk = copy(x)
    for i = 1:maxits
        rk = r(xk)
        Ak = A(xk)
        bk = Ak*xk - rk
        
        ∇f = Ak'*rk
        
        trace = [trace; [norm(∇f) norm(x-xk)]]
        norm(∇f) ≤ tol && norm(x-xk) ≤ tol && break
        x = copy(xk)        
        xk = Ak\bk
    end
    return (xk,trace)
end;

r (generic function with 1 method)

In [69]:
x0 = [mean(df.x), mean(df.y), 100]
x, trace = gn(r, A, x0, maxits=100)
plot(
    plot(trace[:,1],yaxis=:log),
    plot(trace[2:end,2],yaxis=:log)
    )

UndefVarError: UndefVarError: trace not defined

3-element Array{Float64,1}:
 227.6095238095238
 152.2031746031746
 100.0            